In [24]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
from selenium import webdriver
import time
from lxml import html
from random import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [32]:
res = requests.get("https://free-proxy-list.net")
proxies = pd.read_html(res.text)
proxies = proxies[0][:-1]
proxies.columns = [c.replace(' ', '_') for c in proxies.columns]
ip = list(proxies.IP_Address)
port = list(map(str, list(map(int, list(proxies.Port)))))
print(ip)
print(port)

['131.161.47.34', '182.23.79.68', '144.91.116.171', '125.26.7.124', '103.127.189.13', '128.199.217.244', '136.25.2.43', '14.207.58.129', '118.173.232.61', '118.175.207.129', '78.60.203.75', '173.249.44.18', '128.199.227.116', '181.196.151.82', '139.99.105.186', '189.9.55.7', '80.32.122.118', '118.200.73.124', '122.144.4.106', '188.156.240.240', '124.41.211.188', '159.138.34.123', '157.230.45.27', '45.115.63.78', '46.146.209.132', '43.248.24.158', '46.39.41.21', '196.52.80.146', '109.122.80.234', '47.52.231.140', '202.137.10.179', '174.127.155.118', '186.46.136.70', '196.52.65.120', '203.77.249.73', '191.5.0.79', '150.129.250.28', '95.170.200.138', '110.232.74.233', '47.75.11.94', '196.3.97.71', '77.221.220.133', '202.29.218.114', '176.226.251.131', '178.128.108.108', '118.172.201.105', '94.232.11.178', '178.182.228.69', '196.54.30.80', '50.246.120.125', '139.59.101.223', '207.154.231.217', '138.68.161.60', '139.59.109.156', '178.62.193.19', '188.226.141.211', '119.81.199.82', '88.198.2

In [ ]:
category_data = []
localite_data = []
criterion_data = []
price_data = []
monthly_data = []
description_data = []
plus_data = []
energie_data = []

# PROXY = "127.0.0.1:9150"

PROXY = ip[0] + ":" + port[0]

webdriver.DesiredCapabilities.FIREFOX['proxy']={
    "sslProxy":PROXY,
    "ftpProxy":PROXY,
    "socksProxy":PROXY,
    "socksVersion":5,
    "proxyType":"MANUAL"
}

# headless_proxy = "127.0.0.1:3128"
# proxy = Proxy({
#     'proxyType': ProxyType.MANUAL,
#     'httpProxy': headless_proxy,
#     'ftpProxy' : headless_proxy,
#     'sslProxy' : headless_proxy,
#     'noProxy'  : ''
# })

# chrome_options = webdriver.ChromeOptions()

# disable images to speed up the page loading
# prefs = {"profile.managed_default_content_settings.images": 2}
# chrome_options.add_experimental_option("prefs", prefs)

# capabilities = dict(DesiredCapabilities.CHROME)
# proxy.add_to_capabilities(capabilities)

# driver = webdriver.Chrome(desired_capabilities=capabilities, options=chrome_options)
# driver.set_page_load_timeout(600)

max = 20
min = 11

page_num1=57  #20:24

page_num2=page_num1+1
for num in range(page_num1,page_num2):
    driver = webdriver.Firefox()
    
    # driver = webdriver.Chrome(desired_capabilities=capabilities, options=chrome_options)
    # driver.set_page_load_timeout(600)
    cond = 0
    proxy_id = 1
    while(cond == 0):
        try:
            driver.get("https://seloger.com/list.htm?projects=2%2C5&types=1%2C2%2C4%2C3%2C9%2C13%2C14%2C12%2C11%2C10&natures=1%2C2%2C4&places=%5B%7BidPays%3A250%7D%5D&enterprise=0&houseboat=1&qsVersion=1.0&LISTING-LISTpg="+str(num))
            cond = 1
        except:
            driver.quit()
            PROXY = ip[proxy_id] + ":" + port[proxy_id]
            webdriver.DesiredCapabilities.FIREFOX['proxy']={
                "sslProxy":PROXY,
                "ftpProxy":PROXY,
                "socksProxy":PROXY,
                "socksVersion":5,
                "proxyType":"MANUAL"
            }
            driver = webdriver.Firefox()
            proxy_id = proxy_id+1
            print("Change proxy1: " + PROXY)
            
    time_random1 = random()
    time1 = min + (time_random1 * (max - min))
    time.sleep(time1)
    url2 = str(driver.current_url)
    if('validate.perfdrive.com' in url2):
        continue
    page = soup(driver.page_source, 'html.parser')
    house_containers = page.find_all("div", class_="block__ShadowedBlock-sc-10w6hsj-0 ListContent__SmartClassifiedExtended-sc-1viyr2k-2 iddbNe classified__ClassifiedContainer-sc-1wmlctl-0 cGCTai Card__CardContainer-sc-7insep-7 kjDrzL")
    print(len(house_containers))
    
    driver2 = webdriver.Firefox()
    
    # driver2 = webdriver.Chrome(desired_capabilities=capabilities, options=chrome_options)
    # driver2.set_page_load_timeout(600)
    proxy_id2 = 1
    for data in house_containers:
        find_link = data.find_all("a", class_="CoveringLink-a3s3kt-0 dXJclF")
        L=str(find_link[0]).split(" ")
        ad_link=L[3][14:-1]
               
        if ('seloger.com' not in ad_link):
            continue
        
        cond2 = 0
        while(cond2 == 0):
            try:
                driver2.get("https://" + ad_link)
                cond2 = 1
            except:
                driver2.quit()
                PROXY = ip[proxy_id2] + ":" + port[proxy_id2]
                webdriver.DesiredCapabilities.FIREFOX['proxy']={
                    "sslProxy":PROXY,
                    "ftpProxy":PROXY,
                    "socksProxy":PROXY,
                    "socksVersion":5,
                    "proxyType":"MANUAL"
                }
                driver2 = webdriver.Firefox()
                proxy_id2 = proxy_id2+1
                print("Change proxy2: " + PROXY)

        time_random2 = random()
        time2 = min + (time_random2 * (max - min))
        time.sleep(time2)
       
        url = str(driver2.current_url)
       
        if('validate.perfdrive.com' in url): 
            driver2.quit()
            proxy_id2 +=1 
            PROXY = ip[proxy_id2] + ":" + port[proxy_id2]
            webdriver.DesiredCapabilities.FIREFOX['proxy']={
                    "sslProxy":PROXY,
                    "ftpProxy":PROXY,
                    "socksProxy":PROXY,
                    "socksVersion":5,
                    "proxyType":"MANUAL"
            }
            driver2 = webdriver.Firefox()
            # driver2 = webdriver.Chrome(desired_capabilities=capabilities, options=chrome_options)
            # driver2.set_page_load_timeout(600)      
            continue
            
        page2 = soup(driver2.page_source, 'html.parser')

        criterion = []
        plus = []
        energie =[]
        try:
            try:
                try:
                    find_category = page2.find_all("h2", class_="c-h2")[0]
                    category = str(find_category)[len('<h2 class="c-h2">'):-5]
                except IndexError:
                    find_category = page2.find_all("div", class_="Summary__Text-sc-1wkzvu-6 gcWjRm")[0]
                    category=(str(find_category)[len('<div class="Summary__Text-sc-1wkzvu-6 gcWjRm">'):-6])
            except:
                find_category = page2.find_all("div", class_="Summarystyled__Title-tzuaot-3 dkepid")[0]
                category=(str(find_category)[len('<div class="Summarystyled__Title-tzuaot-3 dkepid">'):-6])
        except:
            category = "NULL"
        
        try:
            try:
                try:
                    find_localite = page2.find_all("p", class_="localite")[0]
                    localite = str(find_localite)[len('<p class="localite">'):-4]
                except IndexError:
                    find_localite = page2.find_all("div", class_="Summary__Text-sc-1wkzvu-6 gcWjRm")[1]
                    localite=(str(find_localite)[len('<div class="Summary__Text-sc-1wkzvu-6 gcWjRm">'):-6])
            except:
                find_localite = page2.find_all("div", class_="Summarystyled__Address-tzuaot-5 fLUFm")[0]
                localite=(str(find_localite)[len('<div class="Summarystyled__Address-tzuaot-5 fLUFm">'):-6])
        except:
            localite = "NULL"

        try:
            try:
                find_criterion = page2.find_all("ul", class_="criterion")[0]
                criterion2 = str(find_criterion).split('<li>')[1:]
                for i in range (len(criterion2)):
                    ch=""
                    for j in range (len(criterion2[i])):
                        if(str(criterion2[i])[j]=="<"):
                            break
                        ch=ch+str(criterion2[i])[j]
                    criterion.append(ch)

            except IndexError:
                find_criterion = page2.find_all("div", class_="Tags__TagContainer-sc-1gccxml-0 kQEoGa")[:3]
                find_criterion = list(map(str, find_criterion))
                for i in range (len(find_criterion)):
                    criterion.append(find_criterion[i][len('<div class="Tags__TagContainer-sc-1gccxml-0 kQEoGa">'):-6])

        except:
            criterion.append("NULL")

        try:
            try:
                try:
                    find_price = page2.find_all("a", class_="js-smooth-scroll-link price")[0]
                    price = str(find_price)[len('<a class="js-smooth-scroll-link price" data-smoothscroll-offset="160" href="#about_price_anchor">'):-4].replace(" ","").replace("\n","")
                except IndexError:
                    find_price = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY")[0]
                    price = str(find_price)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY">'):-7]
            except:
                find_price = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN")[0]
                price = str(find_price)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">'):-7]
        except:
            price = "NULL"

        try:
            try:
                try:
                    find_monthly = page2.find_all("a", class_="monthly")[0]
                    monthly = str(find_monthly)[len('<a class="monthly" href="https://e-immobilier.credit-agricole.fr/simulca/?ORI=selogercampagne&amp;xtor=AL-2-[partenariat]-[Seloger]-[pdd]-[lien_a_partir_sticky]" rel="nofollow" target="_blank">'):-len('<sup>1</sup></a>')]
                except IndexError:
                    find_monthly = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY")[1]
                    monthly = str(find_monthly)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-7 iAJdaY">'):-7]
            except:
                find_monthly = page2.find_all("span", class_="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN")[1]
                monthly = str(find_monthly)[len('<span class="global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">'):-7]
        except:
            monthly = "NULL"

        try:
            try:
                find_description = page2.find_all("p", class_="sh-text-light")[0]
                description = str(find_description)[len('<p class="sh-text-light" id="js-descriptifBien">'):-4]
            except IndexError:
                find_description = page2.find_all("div", class_="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd")[0]
                description = str(find_description)[len('<div class="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd">'):-len('<button class="ContactLink__StyledButton-sc-1ex39dt-0 jPagdq">Demander plus d\'informations à l\'agence<div class="ContactLink__Chevron-sc-1ex39dt-1 gYbUwK Icon__IconElement-sc-1xnn3zd-0 fFMQXr" color="#e00034" data-test="contact-link-chevron" style="margin-left:5px"><svg viewbox="0 0 512 512"><path d="M144.9 495.9c-9 0-16.4-7.3-16.4-16.4 0-4.2 1.6-8.3 4.6-11.4L338 256 133.1 43.8c-6.2-6.5-6-16.9.5-23.2 6.5-6.2 16.8-6 23 .4l226.9 235-226.8 234.9c-3.1 3.2-7.4 5-11.8 5z"></path></svg></div></button>')]
        except:
            description = "NULL"

        try:
            try:
                find_plus = page2.find_all("div", class_="u-left")[0]
                find_plus = page2.find_all("div", class_="u-left")
                for i in range (len(find_plus)):
                    plus2 = str(find_plus[i]).replace('<div class="u-left">', "").replace("</div>", "")
                    plus.append(plus2)

            except IndexError:
                find_plus = page2.find_all("figcaption") + page2.find_all("li", class_="GeneralList__Item-sc-9gtpjm-1 dsevYN")
                for i in range (len(find_plus)):
                    ch=""
                    for j in range (len(str(find_plus[i]))):
                        if(str(find_plus[i])[j]=='>'):
                            k=j
                            while(str(find_plus[i])[k]!="<"):
                                ch=ch + str(find_plus[i])[k]
                                k=k+1
                            break
                    plus.append(ch[1:])
        except:
            plus.append("NULL")

        try:
            try:
                find_energie = page2.find_all("div", class_="info-detail")[0]
                find_energie = page2.find_all("div", class_="info-detail")

                energie.append(str(find_energie[0])[len('<div class="info-detail">'):-6])
                energie.append(str(find_energie[1])[len('<div class="info-detail">'):-6])

            except IndexError:
                find_energie = page2.find_all("span", class_="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO")
                energie.append(str(find_energie[0])[len('<span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">'):-7])
                energie.append(str(find_energie[1])[len('<span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">'):-7])
        except:
            energie.append("NULL")
            energie.append("NULL")


        print(category)
        print(localite)
        print(criterion)
        print(price)
        print(monthly)
        print(description)
        print(plus)
        print(energie)
        print(num)
        print("-----------------------------------------------------")

        category_data.append(str(category))
        localite_data.append(str(localite))
        criterion_data.append(criterion)
        price_data.append(str(price))
        monthly_data.append(str(monthly))
        description_data.append(str(description))
        plus_data.append(plus)
        energie_data.append(energie)

        # driver2.quit()
        
    for i in range (len(criterion_data)):
        if (len(criterion_data[i])==1):
            if("pièce" in criterion_data[i][0]):
                criterion_data[i].insert(1, "NULL")
                criterion_data[i].insert(2, "NULL")
            if("chambre" in criterion_data[i][0]):
                criterion_data[i].insert(0, "NULL")
                criterion_data[i].insert(2, "NULL")
            if("m²" in criterion_data[i][0]):
                criterion_data[i].insert(0, "NULL")
                criterion_data[i].insert(1, "NULL")
        if (len(criterion_data[i])==2):
            if(("pièce" in criterion_data[i][0]) and ("chambre" in criterion_data[i][1])):
                criterion_data[i].insert(2, "NULL")
            if(("pièce" in criterion_data[i][0]) and ("m²" in criterion_data[i][1])):
                criterion_data[i].insert(1, "NULL")
            if(("chambre" in criterion_data[i][0]) and ("m²" in criterion_data[i][1])):
                criterion_data[i].insert(0, "NULL")
        if(len(criterion_data[i])==0):
            criterion_data[i].append("NULL")
            criterion_data[i].append("NULL")
            criterion_data[i].append("NULL")
        
    df = pd.DataFrame()
    df["category"] = category_data
    df["localite"] = localite_data
    df["piece"] = [i[0] for i in criterion_data]
    df["chambres"] = [i[1] for i in criterion_data]
    df["area"] = [i[2] for i in criterion_data]
    df["price"] = price_data
    df["monthly"] = monthly_data
    df["description"] = description_data
    df["Diagnostic de performance énergétique(kWhEP/m².an)"] = [i[0] for i in energie_data]
    df["Indice d'émission de gaz à effet de serre(kgeqCO2/m².an)"] = [i[1] for i in energie_data]
    df["plus"] = plus_data
    df["type"] = "Acheter"
    
    df.to_csv(r"C:\Users\admin\Desktop\9raya\page.csv", index=False)
    csv1 = pd.read_csv(r"C:\Users\admin\Desktop\9raya\page.csv")
    csv2 = pd.read_csv(r"C:\Users\admin\Desktop\9raya\full_data.csv")
    full_data = pd.concat([csv1, csv2])
    full_data.to_csv(r"C:\Users\admin\Desktop\9raya\full_data.csv", index=False)
    print('**DONE**')


Change proxy1: 182.23.79.68:80
Change proxy1: 144.91.116.171:443
Change proxy1: 125.26.7.124:61642


In [10]:
driver.quit()
driver2.quit()

In [31]:
csv2 = pd.read_csv(r"C:\Users\admin\Desktop\9raya\full_data.csv")
print(csv2.shape)

(806, 12)
